In [15]:
import numpy as np
import os
from scipy.special import expit
from scipy.stats import norm

In [16]:
# 寻找最优旋转
def find_optimal_rotation_matrix(true_theta, pred_theta):
    pred_theta = pred_theta - pred_theta.mean(axis=0) # 进行中心化
    assert true_theta.shape == pred_theta.shape, "Matrices must have the same shape"
    U, _, Vt = np.linalg.svd(pred_theta.T @ true_theta)
    Q = Vt.T @ U.T 
    return Q

# 对数似然函数的一阶导数
def log_likelihood_first_derivative(A_ij, pi_ij):
    return A_ij - expit(pi_ij)

# 对数似然函数的二阶导数
def log_likelihood_second_derivative(M):
    sigma_ij = expit(M)
    return sigma_ij * (sigma_ij - 1)

# 计算距离
def Distance_matrix(Theta):
    diff = Theta[:, np.newaxis, :] - Theta[np.newaxis, :, :]
    D = np.sum(diff**2, axis=2)
    return D

# 生成M矩阵
def generate_M_matrix(thetas, rho=0):
    M = rho-Distance_matrix(thetas)
    return M

# 计算\hat{\sigma}_i
def compute_sigma_hat(i, M, Z, r=2):
    sigma_hat_i = np.zeros((r, r))
    for j in range(M.shape[0]):
        if i != j:
            h_i = Z[i].reshape(-1, 1)
            h_j = Z[j].reshape(-1, 1)
            l_double_prime = log_likelihood_second_derivative(M[i, j])
            sigma_hat_i += 4 * l_double_prime * (h_j-h_i) @ (h_j-h_i).T
    sigma_hat_i /= M.shape[0]
    return sigma_hat_i

# 计算指定的\hat{\Omega}_i
def compute_omega_hat(i, M, Z, A, r=2):
    omega_hat_i = np.zeros((r, r))
    for j in range(M.shape[0]):
        if i != j:
            h_i = Z[i].reshape(-1, 1)
            h_j = Z[j].reshape(-1, 1)
            l_double_prime = log_likelihood_first_derivative(A[i,j], M[i,j])**2
            omega_hat_i += 4 * l_double_prime *  (h_j-h_i) @ (h_j-h_i).T
    omega_hat_i /= M.shape[0]
    return omega_hat_i

#### 计算样本协方差矩阵

In [17]:
number = 500
base_folder = r'/home/user/CYH/Code_For_MDS/Project/para_result/parameter estimation/distance1/Binomial/Simulation10000_2_1.0relative_1e-08'
true_theta = np.load(base_folder + f'/n_{number}/1/true_theta.npy')

r = 2
H = []
path_ = base_folder + f'/n_{number}/'
for _ in range(1,len(os.listdir(path_ ))+1):
    pred_theta = np.load(path_+f'/{_}/pred_theta.npy')
    Q = find_optimal_rotation_matrix(true_theta, pred_theta)
    pred_theta = pred_theta @ Q.T
    H.append(pred_theta[0])
H = np.array(H)
true_matrix = np.cov(H, rowvar=False)
true_matrix

array([[8.67644404e-03, 9.97118081e-05],
       [9.97118081e-05, 7.48495418e-03]])

#### 计算sandwich的值

In [18]:
haha = {}
for _ in range(1,201):
    A = np.load(base_folder + f'/n_{number}/{_}/adjacency_matrix.npy')
    pred_theta = np.load(base_folder + f'/n_{number}/{_}/pred_theta.npy')
    Q = find_optimal_rotation_matrix(true_theta, pred_theta)
    pred_theta = pred_theta @ Q.T
    M = generate_M_matrix(pred_theta)

    left =compute_sigma_hat(0, M, pred_theta)
    mid = compute_omega_hat(0, M, pred_theta, A)
    a = np.linalg.inv(left)
    sandwich = a@mid@a

    
    haha[_] = np.sum(np.abs(sandwich/number - true_matrix))

In [19]:
# 找寻表现最好的值
min_key = min(haha, key=haha.get)
max_key = max(haha, key=haha.get)
min_key, max_key

(179, 30)

In [20]:
_ = 179
A = np.load(base_folder + f'/n_{number}/{_}/adjacency_matrix.npy')
pred_theta = np.load(base_folder + f'/n_{number}/{_}/pred_theta.npy')
Q = find_optimal_rotation_matrix(true_theta, pred_theta)
pred_theta = pred_theta @ Q.T
M = generate_M_matrix(pred_theta)

left =compute_sigma_hat(0, M, pred_theta)
mid = compute_omega_hat(0, M, pred_theta, A)
a = np.linalg.inv(left)
sandwich = a@mid@a
sandwich/number

array([[8.49474058e-03, 6.19492030e-05],
       [6.19492030e-05, 7.30893440e-03]])

#### coverage rates

In [21]:
def calculate_sandwich(base_path):
    A = np.load(f'{base_path}/adjacency_matrix.npy')
    pred_theta = np.load(f'{base_path}/pred_theta.npy')
    true_theta = np.load(f'{base_path}/true_theta.npy')
    
    Q = find_optimal_rotation_matrix(true_theta, pred_theta)
    pred_theta = pred_theta @ Q.T

    M = generate_M_matrix(pred_theta)

    sigma_hat = compute_sigma_hat(0, M, pred_theta)
    omega_hat = compute_omega_hat(0, M, pred_theta, A)
    a = np.linalg.inv(sigma_hat)
    sandwich = a @ omega_hat @ a
    sandwich = sandwich/pred_theta.shape[0]

    return sandwich[0][0], pred_theta[0][0], true_theta[0][0]

def check_confidence_interval(base_path):
    var_estimate, pred_estimate, true_value = calculate_sandwich(base_path)
    standard_error = np.sqrt(var_estimate)
    
    z_score = norm.ppf(0.975)
    lower_bound = pred_estimate - z_score * standard_error
    upper_bound = pred_estimate + z_score * standard_error
    
    is_within_interval = lower_bound <= true_value <= upper_bound
    
    return is_within_interval

# coverage rates 
base_folder = r'/home/user/CYH/Code_For_MDS/Project/para_result/parameter estimation/distance1/Binomial/Simulation10000_2_1.0relative_1e-08'
n_values = sorted([int(i.split('_')[-1]) for i in os.listdir(base_folder) if i.startswith('n_')])
coverage_dict = {}

for number in n_values:
    path = os.path.join(base_folder, f'n_{number}')
    __ = 0
    for _ in range(len(os.listdir(path))):
        if check_confidence_interval(path+f'/{_+1}'):
            __ += 1
    coverage_dict[number] = __/len(os.listdir(path))
    
coverage_dict

KeyboardInterrupt: 